地学数据三维可视化课件第六节：vedo

# <center>地学数据三维可视化-vedo</center>
<br/>

[<center><img src="./fig/geo.gif" style="float: center; height:280px" /><center>](https://vedo.embl.es/examples/geo_scene.html)
<br/>

<div style="clear: both"></div>
<center>汪宇锋</center>
<div style="clear: both"></div>
<center>地球物理与空间信息学院</center>
<div style="clear: both"></div>
<center>2021年11月</center>

## vedo
<br/>

<table>
  <tr>
    <td><img src="./fig/gyroid.gif" width="500" /></td>
    <td><img src="./fig/code_gyroid.png" width="810" /></td>
  </tr>
 </table>


## Quick Start
<br/>

Inspired by the [`vpython`](https://vpython.org/) manifesto “3D programming for ordinary mortals”, vedo makes it easy to work wth three-dimensional objects, create displays and animations in just a few lines of code, even for less experienced programmers. vedo is based on **VTK** and **numpy**, with no other dependencies.

```python
pip install vedo
vedo https://vedo.embl.es/examples/geo_scene.npz
```

Check out the Git repository: https://github.com/marcomusy/vedo

## Utah Forge

This is a notebook showing how to recreate a model of a geothermal reservoir using publicly available data. The model is of a reservoir in Utah that is part of a project called FORGE (Frontier Observatory for Research in Geothermal Energy).

See image that you will generate here: https://github.com/ahinoamp/Example3DGeologicModelUsingVTKPlotter/blob/master/ModelImage.png

Original data source links are shown in the end.

In [1]:
#imports
from vedo import *
import pandas as pd
settings.useDepthPeeling = True    
embedWindow('k3d') # k3d, panel or itkwidgets or False (for a popup)

In [2]:
#########################
# Load surfaces, import the file from github
#########################
printc("...locating data source...", invert=1)

# url = "https://raw.githubusercontent.com/ahinoamp/Example3DGeologicModelUsingVTKPlotter/master/"
url = './data/3DGeologicModel/'

...locating data source...


In [3]:
#Load surfaces, fault, and microseismic data
printc("...loading surfaces data...", invert=1)

landSurfacePD = pd.read_csv(url+"land_surface_vertices.csv")
vertices_175CPD = pd.read_csv(url+"175C_vertices.csv")
vertices_225CPD = pd.read_csv(url+"225C_vertices.csv")
Negro_Mag_Fault_verticesPD = pd.read_csv(url+"Negro_Mag_Fault_vertices.csv")
Opal_Mound_Fault_verticesPD = pd.read_csv(url+"Opal_Mound_Fault_vertices.csv")
top_granitoid_verticesPD = pd.read_csv(url+"top_granitoid_vertices.csv")
microseismic = pd.read_csv(url+"Microseismic.csv")

...loading surfaces data...


In [4]:
# The well path and different logs for the well paths
printc("...loading well data...", invert=1)

well_5832_path = pd.read_csv(url+"path5832.csv")
temp_well = pd.read_csv(url+"temperature5832.csv")
nphi_well = pd.read_csv(url+"nphi5832.csv")
pressure_well = pd.read_csv(url+"pressure5832.csv")
# Since most of the wells in the area were just vertical, I split them into two files:
# One file with the top of the wells and the other with the bottom point of the wellbore
wellsmin = pd.read_csv(url+"MinPointsWells.csv")
wellsmax = pd.read_csv(url+"MaxPointsWells.csv")

...loading well data...


In [5]:
# Project boundary area on the surface
printc("...loading site boundry data...", invert=1)

border = pd.read_csv(url+"FORGE_Border.csv")

...loading site boundry data...


In [6]:
#############################################
## 1. land surface: a mesh with varying color
#############################################
printc("...plotting surfaces data...", invert=1, end='')

# create a mesh object from the 2D Delaunay triangulation of the point cloud
landSurface = delaunay2D(landSurfacePD.values)

# in order to color it by the elevation, we use the z values of the mesh
zvals = landSurface.points()[:, 2]
landSurface.cmap("terrain", zvals, vmin=1100)
landSurface.name = "Land Surface" # give the object a name

# Create a plotter and add landSurface to it
plt = Plotter(axes=dict(xtitle='km', ytitle=' ', ztitle='km*1.5', yzGrid=False),
              bg2='lb', size=(1200,900)) # screen size
plt += landSurface.flag()                # this adds a flag when hoovering the mouse
plt += landSurface.isolines(5).lw(1).c('r')

...plotting surfaces data...

In [7]:
plt.show(viewup="z")

Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0…

In [8]:
## Different meshes with constant colors
printc("...plotting isotherm surfaces...", invert=1, end='')

# Mesh of 175 C isotherm
vertices_175C = delaunay2D(vertices_175CPD.values)
vertices_175C.name = "175C temperature isosurface"
plt += vertices_175C.c("orange").opacity(0.3).flag()

# Mesh of 225 C isotherm
vertices_225CT = delaunay2D(vertices_225CPD.values)
vertices_225CT.name = "225C temperature isosurface"
plt += vertices_225CT.c("r").opacity(0.4).flag()

...plotting isotherm surfaces...

In [9]:
plt.show(viewup="z")

Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0…

In [10]:
printc("...plotting fault surfaces...", invert=1, end='')

# Negro fault, mode=fit is used because point cloud is not in xy plane
Negro_Mag_Fault_vertices = delaunay2D(Negro_Mag_Fault_verticesPD.values, mode='fit')
Negro_Mag_Fault_vertices.name = "Negro Fault"
plt += Negro_Mag_Fault_vertices.c("f").opacity(0.6).flag()

# Opal fault
Opal_Mound_Fault_vertices = delaunay2D(Opal_Mound_Fault_verticesPD.values, mode='fit')
Opal_Mound_Fault_vertices.name = "Opal Mound Fault"
plt += Opal_Mound_Fault_vertices.c("g").opacity(0.6).flag()

...plotting fault surfaces...

In [11]:
plt.show(viewup="z")

Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0…

In [12]:
# Top Granite, (shift it a bit to avoid overlapping)
printc("...plotting Top Granite surface...", invert=1, end='')
xyz = top_granitoid_verticesPD.values - [0,0,20]
top_granitoid_vertices = delaunay2D(xyz).texture('paper2')
top_granitoid_vertices.name = "Top of granite surface"
plt += top_granitoid_vertices.flag()

...plotting Top Granite surface...

In [13]:
plt.show(viewup="z")

Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0…

In [14]:
printc("...plotting Microseismic data...", invert=1)

# Microseismic
microseismicxyz = microseismic[["xloc", "yloc", "zloc"]].values
scals = microseismic[["mw"]]
microseismicPts = Points(microseismicxyz, r=5).cmap("jet", scals)
microseismicPts.name = "Microseismic events"
plt += microseismicPts.flag()

...plotting Microseismic data...


In [15]:
plt.show(viewup="z")

Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0…

In [16]:
# FORGE Boundary. Since the boundary area did not have a Z column,
# I assigned a Z value for where I wanted it to appear
printc("...plotting site boundary...", invert=1)

border["zcoord"] = 1650
borderxyz = border[["xcoord", "ycoord", "zcoord"]]
boundary = Line(borderxyz.values).extrude(zshift=120, cap=False).lw(0).texture('wood1')
boundary.name = "FORGE area boundary"
plt += boundary.flag()

...plotting site boundary...


In [17]:
plt.show(viewup="z")

Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0…

In [18]:
# The path of well 58_32
Well1 = Line(well_5832_path[["X", "Y", "Z"]].values, lw=2, c='k')
Well1.name = "Well 58-32"
plt += Well1.flag()

In [19]:
plt.show(viewup="z")

Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0…

In [20]:
# A porosity log in the well
xyz = nphi_well[["X", "Y", "Z"]].values
porosity = nphi_well["Nphi"].values
Well2 = Line(xyz, lw=3).cmap("hot", porosity)
Well2.name = "Porosity log well 58-32"
plt += Well2.flag()

In [21]:
plt.show(viewup="z")

Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0…

In [22]:
# This well data is actually represented by points since as of right now,
xyz = pressure_well[["X", "Y", "Z"]].values
pressure = pressure_well["Pressure"].values
Well3 = Line(xyz, lw=3).cmap("cool", pressure)
Well3.name = "Pressure log well 58-32"
plt += Well3.flag()

In [23]:
plt.show(viewup="z")

Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0…

In [24]:
# Temperature log
xyz = temp_well[["X", "Y", "Z"]].values
temp = temp_well["Temperature"].values
Well4 = Line(xyz, lw=3).cmap("seismic", temp)
Well4.name = "Temperature log well 58-32"
plt += Well4.flag()

In [25]:
plt.show(viewup="z")

Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0…

In [26]:
# defining the start and end of the lines that will be representing the wellbores
Wells = Lines(wellsmin[["x", "y", "z"]].values, # start points
              wellsmax[["x", "y", "z"]].values, # end points
              c="gray", alpha=1, lw=3)
Wells.name = "Pre-existing wellbores"
plt += Wells.flag()

In [27]:
plt.show(viewup="z")

Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0…

In [28]:
printc("...plotting the final results...", invert=1)

#for a in plt.actors:
#    # change scale to kilometers in x and y, but expand z scale by 1.5!
#    a.scale([0.001, 0.001, 0.001*1.5])

#########################
## show the plot
plt += __doc__
plt.show(viewup="z", zoom=1.2)

...plotting the final results...


Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0…

In [29]:
printc("...plotting the final results...", invert=1)

plt.export("page.html") # k3d is the default

...plotting the final results...


In [30]:
printc("...removing sth from the final results...", invert=1)
#plt -= Wells.flag()

plt.show(viewup="z", zoom=1.2)

...removing sth from the final results...


Plot(antialias=True, axes=['x', 'y', 'z'], axes_helper=1.0, background_color=16777215, camera=[2, -3, 0.2, 0.0…

## vedo with [dolfin](https://fenicsproject.org/)

- FEniCS is a popular open-source (LGPLv3) computing platform for solving partial differential equations (PDEs). FEniCS enables users to quickly translate scientific models into efficient finite element code. 

- With the high-level Python and C++ interfaces to FEniCS, it is easy to get started, but FEniCS offers also powerful capabilities for more experienced programmers. 

- FEniCS runs on a multitude of platforms ranging from laptops to high-performance clusters.

In [31]:
!source /Users/wangyufeng/opt/anaconda3/share/dolfin/dolfin.conf
!python ./data/awefem.py

Traceback (most recent call last):
  File "./data/awefem.py", line 9, in <module>
    from dolfin import *
  File "/Users/wangyufeng/opt/anaconda3/lib/python3.8/site-packages/dolfin/__init__.py", line 142, in <module>
    from .fem.assembling import (assemble, assemble_system, assemble_multimesh,
  File "/Users/wangyufeng/opt/anaconda3/lib/python3.8/site-packages/dolfin/fem/assembling.py", line 34, in <module>
    from dolfin.fem.form import Form
  File "/Users/wangyufeng/opt/anaconda3/lib/python3.8/site-packages/dolfin/fem/form.py", line 12, in <module>
    from dolfin.jit.jit import dolfin_pc, ffc_jit
  File "/Users/wangyufeng/opt/anaconda3/lib/python3.8/site-packages/dolfin/jit/jit.py", line 18, in <module>
    raise RuntimeError("Could not find DOLFIN pkg-config file. Please make sure appropriate paths are set.")
RuntimeError: Could not find DOLFIN pkg-config file. Please make sure appropriate paths are set.


# 下节课预告
<br/>

<center class="half">
    <img src="./fig/ParaView-MoshaFault.gif" style="float: center; width:600px"/> 
</center>  